# Understanding the Dataset #

In [33]:
from sklearn.datasets import load_iris
import numpy as np

iris_data = load_iris()
features = iris_data.data
labels = iris_data.target

feature_labels = iris_data.feature_names
class_names = iris_data.target_names

print("Features available:", ", ".join(feature_labels))
print("Classes to predict:", class_names)
print("Feature matrix dimensions:", features.shape)
print("Label vector length:", labels.shape)

Features available: sepal length (cm), sepal width (cm), petal length (cm), petal width (cm)
Classes to predict: ['setosa' 'versicolor' 'virginica']
Feature matrix dimensions: (150, 4)
Label vector length: (150,)


# Building the Decision Tree #

* Node Structure 

In [34]:
class TreeNode:
    def __init__(
        self,
        split_feature=None,
        split_threshold=None,
        left_child=None,
        right_child=None,
        predicted_value=None,
    ):
        self.split_feature = split_feature
        self.split_threshold = split_threshold
        self.left_child = left_child
        self.right_child = right_child
        self.predicted_value = predicted_value

* Splitting Criteria (Gini Impurity)

In [35]:
def compute_gini_impurity(labels):
    unique_classes = np.unique(labels)
    impurity = 1.0  # Start with maximum impurity

    for current_class in unique_classes:
        class_prob = np.mean(labels == current_class)
        impurity -= class_prob**2

    return impurity


def find_optimal_split(features, labels):
    n_samples, n_features = features.shape

    # Edge case: can't split if there's only one sample
    if n_samples <= 1:
        return None, None

    current_impurity = compute_gini_impurity(labels)
    best_feature_idx, best_split_value = None, None

    for feature_idx in range(n_features):
        possible_thresholds = np.unique(features[:, feature_idx])

        for threshold in possible_thresholds:
            # Split data into left/right nodes
            left_mask = features[:, feature_idx] <= threshold
            right_mask = features[:, feature_idx] > threshold

            # all samples go to one side
            if np.sum(left_mask) == 0 or np.sum(right_mask) == 0:
                continue

            left_impurity = compute_gini_impurity(labels[left_mask])
            right_impurity = compute_gini_impurity(labels[right_mask])

            # weighted average impurity
            left_weight = np.sum(left_mask) / n_samples
            combined_impurity = (
                left_weight * left_impurity + (1 - left_weight) * right_impurity
            )

            # Update best split if we found a better one
            if combined_impurity < current_impurity:
                current_impurity = combined_impurity
                best_feature_idx = feature_idx
                best_split_value = threshold

    return best_feature_idx, best_split_value

* Recursive Splitting

In [36]:
class DecisionTree:
    def __init__(self, max_depth=5, min_samples_per_node=2):
        self.max_depth = max_depth
        self.min_samples = min_samples_per_node
        self.root_node = None

    def _grow_tree(self, data, labels, current_depth=0):
        n_samples, n_features = data.shape
        unique_classes = np.unique(labels)

        # Check if we should stop and make a leaf
        if (
            current_depth >= self.max_depth
            or n_samples < self.min_samples
            or len(unique_classes) == 1
        ):
            return TreeNode(predicted_value=self._get_plurality_class(labels))

        best_feat, best_thresh = find_optimal_split(data, labels)

        # If we can't find a good split, make a leaf
        if best_feat is None:
            return TreeNode(predicted_value=self._get_plurality_class(labels))

        # Split data and grow branches
        goes_left = data[:, best_feat] <= best_thresh
        left_branch = self._grow_tree(
            data[goes_left], labels[goes_left], current_depth + 1
        )
        right_branch = self._grow_tree(
            data[~goes_left], labels[~goes_left], current_depth + 1
        )

        return TreeNode(
            split_feature=best_feat,
            split_threshold=best_thresh,
            left_child=left_branch,
            right_child=right_branch,
        )

    def _get_plurality_class(self, labels):
        """Returns the most frequent class in labels (handles ties)."""
        classes, counts = np.unique(labels, return_counts=True)
        return classes[np.argmax(counts)]

    def fit(self, X, y):
        self.root_node = self._grow_tree(X, y)

    def predict(self, X):
        return np.array([self._walk_tree(sample, self.root_node) for sample in X])

    def _walk_tree(self, sample, node):
        if node.predicted_value is not None:
            return node.predicted_value

        if sample[node.split_feature] <= node.split_threshold:
            return self._walk_tree(sample, node.left_child)
        return self._walk_tree(sample, node.right_child)

# Training and Evaluation #

In [37]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    classification_report,
)

X_train, X_test, y_train, y_test = train_test_split(
    features,
    labels,
    test_size=0.2,
    random_state=42,
    stratify=labels,  # to maintain class distribution
)

iris_tree = DecisionTree(max_depth=3)
iris_tree.fit(X_train, y_train)

test_predictions = iris_tree.predict(X_test)

print("\nModel Performance:")
print(f"Overall Accuracy: {accuracy_score(y_test, test_predictions):.3f}")
print(
    f"Avg Precision: {precision_score(y_test, test_predictions, average='macro'):.3f}"
)
print(f"Avg Recall: {recall_score(y_test, test_predictions, average='macro'):.3f}")
print(f"F1 Score: {f1_score(y_test, test_predictions, average='macro'):.3f}")

print("\nDetailed Report:")
print(
    classification_report(y_test, test_predictions, target_names=iris_data.target_names)
)


Model Performance:
Overall Accuracy: 0.967
Avg Precision: 0.970
Avg Recall: 0.967
F1 Score: 0.967

Detailed Report:
              precision    recall  f1-score   support

      setosa       1.00      1.00      1.00        10
  versicolor       1.00      0.90      0.95        10
   virginica       0.91      1.00      0.95        10

    accuracy                           0.97        30
   macro avg       0.97      0.97      0.97        30
weighted avg       0.97      0.97      0.97        30

